In [ ]:
import requests
import urllib.request
import json
import pandas as pd
import time
from datetime import datetime, timedelta
from sqlalchemy import create_engine, event
from urllib.parse import quote_plus
import sys
import datetime as DT
import gspread
from gspread_dataframe import set_with_dataframe
from google.oauth2.service_account import Credentials
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import re

In [ ]:
scopes = ['https://www.googleapis.com/auth/spreadsheets',
          'https://www.googleapis.com/auth/drive']

credentials = Credentials.from_service_account_file('lively-clover-265718-6f7adafc0648.json', scopes=scopes)

gc = gspread.authorize(credentials)

gauth = GoogleAuth()
drive = GoogleDrive(gauth)

In [ ]:
API_KEY = ""
CLIENT_ID = ''
hdrs = {"Client-Id": CLIENT_ID, "Api-Key": API_KEY,"Content-Type": "application/json"}

In [ ]:
#Выгружаем отчет по транзакциям за нужный период

url = "https://api-seller.ozon.ru/v3/finance/transaction/list"

date_from = "2023-05-01T00:00:00.000Z"
date_to = "2023-05-16T00:00:00.000Z"

body = {
        "filter": {
            "date": {
                "from": date_from,
                "to": date_to
            },
            "operation_type": [],
            "posting_number": "",
            "transaction_type": "all"
        },
        "page": 1,
        "page_size": 1000
    }
body = json.dumps(body)

report_data = requests.post(url, headers=hdrs, data=body)
json_report_data = report_data.json()

In [ ]:
report_data.text

In [ ]:

data_to_df = json_report_data["result"]["operations"]
df_5 = pd.json_normalize(data_to_df, errors='ignore', sep=' ', max_level=None)
#df_flatten_1 = pd.json_normalize(data_to_df, errors='ignore', sep=' ', max_level=None,record_path=['items'])
#df_1 = df_1.drop(columns='items')
df_5 = df_5.drop(columns='services')
df_5['items'] = df_5['items'].apply(lambda x: re.sub("[',[,{,:]", " ", str(x)).replace('name','').replace(']','').replace('sku',''))

In [ ]:
finance_report = pd.concat([df_1,df_2,df_3,df_4,df_5]).drop_duplicates()

In [ ]:
finance_report[finance_report.duplicated()]

In [ ]:
#Записываем данные в гугл шит
# open a google sheet
gs = gc.open_by_key('')
# select a work sheet from its name
worksheet1 = gs.worksheet('FINANCE_REPORT')

worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe=finance_report, include_index=False,include_column_header=True, resize=True)

#Отчет по заказам

In [ ]:
url = "https://api-seller.ozon.ru/v1/report/postings/create"
url_2 = "https://api-seller.ozon.ru/v1/report/info"
date_from = "2023-05-01T00:00:00.000Z"
date_to = "2023-05-16T00:00:00.000Z"

body = {
        "filter": {
    "processed_at_from": date_from,
    "processed_at_to": date_to,
    "delivery_schema": [
    "fbs",
    "fbo",
    "crossborder"
],
    "sku": [ ],
    "cancel_reason_id": [ ],
    "offer_id": "",
    "status_alias": [ ],
    "statuses": [ ],
    "title": ""
},
"language": "DEFAULT"
    }

body = json.dumps(body)
report_data = requests.post(url, headers=hdrs, data=body)
json_report_data = report_data.json()

json_report_data



In [ ]:
report_data = requests.post(url_2, headers=hdrs, json=json_report_data['result'])
json_report_data = report_data.json()

In [ ]:
report_data.text

In [ ]:
urllib.request.urlretrieve(json_report_data['result']['file'], "Orders.csv")

In [ ]:
orders_df_5 = pd.read_csv("Orders.csv",sep=';')

orders_df_5

In [ ]:
orders_df_1
orders_df_2
orders_df_3
orders_df_4
orders_df_5

In [ ]:
orders_final = pd.concat([orders_df_1,orders_df_2,orders_df_3,orders_df_4,orders_df_5]).drop_duplicates()

In [ ]:
orders_final[orders_final.duplicated()]

In [ ]:
# open a google sheet
gs = gc.open_by_key('')
# select a work sheet from its name
worksheet1 = gs.worksheet('ORDERS')

In [ ]:
# write to dataframe

worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe=orders_final, include_index=False,include_column_header=True, resize=True)

In [ ]:
finance_report
orders_final

In [ ]:
#Берем нужные колонки из orders
total_orders_1 = orders_final[['Номер отправления','Статус','Артикул','Цена','Количество']]

#Объединяем их с FINANCE
final_finance = finance_report.merge(total_orders_1,how='left',left_on='posting posting_number', right_on='Номер отправления')

In [ ]:
final_finance = final_finance.drop_duplicates().fillna('')

In [ ]:
final_finance[final_finance.duplicated()]

In [ ]:
#Записываем данные в гугл шит
# open a google sheet
gs = gc.open_by_key('')
# select a work sheet from its name
worksheet1 = gs.worksheet('TOTAL_FINANCE')

worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe=final_finance, include_index=False,include_column_header=True, resize=True)